In [ ]:
from sentence_transformers import SentenceTransformer
import lancedb
from tqdm.notebook import tqdm
import torch
import re
import pandas as pd
from pathlib import Path
import json
from datetime import datetime
from dotenv import load_dotenv

from openai import OpenAI
from google import genai
from google.genai import types
from sklearn.metrics.pairwise import cosine_similarity
import os
import time
load_dotenv('env_var')

import wikipediaapi
import wikipedia
from burr.core import action, State, ApplicationBuilder, ApplicationContext, Action
from burr.core.parallelism import MapStates, RunnableGraph

from working_folder.workflows.topic_trend_research.processes.report_prompts import trend_report_prompt, follow_up_prompt, follow_up_query_prompt, \
                                                                                executive_summary_prompt

wiki_wiki = wikipediaapi.Wikipedia(user_agent='Wonky', language='en')

In [ ]:
%load_ext burr.integrations.notebook
%burr_ui

In [ ]:
# index = lancedb.connect('../wonky_data/indexes/')
# table = index.open_table('sections_hybrid')
# encoder = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', device='mps', trust_remote_code=True)

In [ ]:
base_folder = Path('../wonky_data/')
index_folder = base_folder.joinpath('indexes')

# db = lancedb.connect(index_folder)
# report_tbl = db.open_table("reports")
# section_tbl = db.open_table("reports_section")

In [ ]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
def call_llm(query, temperature=0.3, seed=42, model="gemma-3-4b-it@Q8_0"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": query}
        ],
        temperature=temperature,
        seed=seed,
    )
    return completion.choices[0].message.content

model = "gemini-2.0-flash"
total_tokens = list()

def call_llm_flash(query, temperature=0.1, seed=42, max_tokens=7500 ):
    client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])
    response = client.models.generate_content(
        model=model,
        contents=[query],
        config=types.GenerateContentConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
            seed=seed
        )
    )
    total_tokens.append({'prompt_tokens':response.usage_metadata.prompt_token_count,
                         'completion_tokens':response.usage_metadata.candidates_token_count,
                         'total_tokens':response.usage_metadata.total_token_count,
                         'timestamp':datetime.now().strftime("%Y_%m_%d_%H_%M_%S")})

    return response.text

In [ ]:
from copy import deepcopy
class ParseWikipediaSections():
    def __init__(self):
        self.parsed_sections = list()
        self.current_hierarchy = {'level_0':'', 'level_1':'', 'level_2':'', 'level_3':'', 'level_4':''}

    def _recursive_extract(self, sections, level=0):
        for s in sections:
            # print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
            hierarchy_string = list()
            for i in range(0,5):
                if i == level:
                    self.current_hierarchy[f"level_{i}"] = s.title
                elif i > level:
                    self.current_hierarchy[f"level_{i}"] = ''

            for i in range(0,5):
                _hierarchy_header = self.current_hierarchy[f"level_{i}"]
                if _hierarchy_header != "":
                    hierarchy_string.append(_hierarchy_header)
            hierarchy_string = '|'.join(hierarchy_string)

            self.parsed_sections.append({"level": level,
                                         "title": s.title,
                                         "text": s.text,
                                         "hierarchy": deepcopy(self.current_hierarchy),
                                         "hierarchy_string": hierarchy_string})

            self._recursive_extract(s.sections, level + 1)

    def parse_wikipedia_sections(self, sections):
        self._recursive_extract(sections, level=0)
        return self.parsed_sections

def get_wikipedia_data(page_name):
    wikipedia_page = wiki_wiki.page(page_name)
    wikipedia_parser = ParseWikipediaSections()
    parsed_sections = wikipedia_parser.parse_wikipedia_sections(wikipedia_page.sections)
    wikipedia_data = {'title': wikipedia_page.title,
                      'url': wikipedia_page.fullurl,
                      'sections': parsed_sections}
    return wikipedia_data

def search_wikipedia_page(query, num_results=10):
    wiki_wiki = wikipediaapi.Wikipedia(user_agent='Wonky', language='en')
    search_results = wikipedia.search(query, results=num_results, suggestion=False)
    print(search_results)
    wikipedia_page_results = list()
    for search_result in search_results:
        wikipedia_page = wiki_wiki.page(search_result)
        wikipedia_parser = ParseWikipediaSections()
        parsed_sections = wikipedia_parser.parse_wikipedia_sections(wikipedia_page.sections)
        try:
            wikipedia_data = {'title': wikipedia_page.title,
                              'url': wikipedia_page.fullurl,
                              'sections': parsed_sections,
                              'full_text': wikipedia_page.text}
        except Exception as e:
            continue
        wikipedia_page_results.append(wikipedia_data)
        time.sleep(1)
    return wikipedia_page_results

def format_sections_w_citations(wiki_document, base_citation):
    first_section = wiki_document[0]
    current_level_1_idx = 1
    current_level_2_idx = 1
    current_level_1_name = first_section['hierarchy']['level_0']
    current_level_2_name = first_section['hierarchy']['level_1']
    formatted_sections = list()
    first_section['citation'] = f"{base_citation}({current_level_1_idx})({current_level_2_idx})"
    formatted_sections.append(first_section)
    if len(wiki_document) > 1:
        for section in wiki_document[1:]:
            # if current_level_1_name == '':
            #     current_level_1_name = section['hierarchy']['level_0']
            # if current_level_2_name == '':
            #     current_level_2_name = section['hierarchy']['level_1']
            #     current_level_2_idx = 1

            if current_level_1_name != section['hierarchy']['level_0']:
                current_level_1_name = section['hierarchy']['level_0']
                current_level_2_name = section['hierarchy']['level_1']
                current_level_1_idx += 1
                current_level_2_idx = 1
                print(current_level_1_name)
            elif (current_level_1_name == section['hierarchy']['level_0']) and (current_level_2_name != section['hierarchy']['level_1']):
                current_level_2_name = section['hierarchy']['level_1']
                current_level_2_idx += 1
            else:
                current_level_2_idx += 1

            section['citation'] = f"{base_citation}({current_level_1_idx})({current_level_2_idx})"
            print(section['citation'])
            formatted_sections.append(section)
            # current_level_1_idx += 1
            # current_level_2_idx = 1
    return formatted_sections

In [ ]:
report_folder = Path('wonky_data/data/research_data')
report_file_name = "rural broadband in america_20250416150226.json"
research = [report_folder.joinpath(report_file_name)]
# research_project = 'south_america_research_topics'
# report_folder = report_folder.joinpath(research_project)
# research = list(report_folder.glob('*.json'))

context_folder = Path('wonky_data/data/context_research')
context_file = [context_folder.joinpath(report_file_name)]
research, context_file

In [ ]:
with open(research[0],'r') as f:
    research_data = json.load(f)

with open(context_file[0],'r') as f:
    context_data = json.load(f)

### Burr Outline

1. Load the data based on a source file
2. Create the trend analysis report
3. Create the follow up document for additional context
4. Vectorize the wikipedia search results and index them
5. Search for the relevant wikipedia articles
6. Format the returned wikipedia sections
7. Extract new information from wikipedia
8. Create new wikipedia trend report
9. Merge wikipedia trend reports
10. Create an executive summary

In [ ]:
@action(reads=[], writes=["report_data", "file_name", "file_path"])
def load_report_from_file(state: State, file_path: Path) -> State:
    with open(file_path,'r') as f:
        report_data = json.load(f)

    return state.update(report_data=report_data,
                        file_path=str(file_path),
                        file_name=str(file_path.name))

@action(reads=["report_data"], writes=["trend_report"])
def create_trend_analysis(state: State, topic: str, focus:str) -> State:
    formatted_prompt = trend_report_prompt.format(topic=topic, focus=focus, report_data=state['report_data'])
    trend_report = call_llm(formatted_prompt)

    return state.update(trend_report=trend_report)

@action(reads=["trend_report"], writes=["follow_up_report"])
def create_search_follow_up_document(state: State, topic: str, focus: str) -> State:
    formatted_prompt = follow_up_prompt.format(topic=topic, focus=focus, trend_report=state['trend_report'])
    follow_up_report = call_llm(formatted_prompt)
    return state.update(follow_up_report=follow_up_report)

@action(reads=["follow_up_report"], writes=["follow_up_sources"])
def search_wikipedia(state: State) -> State:
    follow_ups = dict()
    for section, follow_ups in state['follow_up_report'].items():
        formatted_follow_up_prompt = follow_up_query_prompt.format(topic=section, focus=section, trend_report=state['trend_report'],
                                                             follow_up=follow_ups)
        query = call_llm(formatted_follow_up_prompt)
        results = search_wikipedia_page(query, num_results=10)

        follow_ups[section] = results

    return state.update(follow_up_sources=follow_ups)

@action(reads=["follow_up_sources"], writes=["temp_index"])
def vectorize_and_index_wikipedia(state: State, temp_index_path: Path) -> State:
    # magic here to create temporary database

    # create the full document index

    # create the section level index

    return state.update(temp_index = temp_index_path)

@action(reads=["temp_index"], writes=["context_sections"])
def search_relevant_wiki_sections(state: State) -> State:
    context_sections = list()
    # magic to search the index

    # Add citations to the information

    return state.update(context_sections=context_sections)

@action(reads=["context_sections"], writes=["wikipedia_trend_report"])
def create_wikipedia_trend_report(state: State, topic: str, focus: str) -> State:
    formatted_trend_prompt = trend_report_prompt.format(topic=topic, focus=focus,
                                                        context_data=state['context_sections'])
    wikipedia_trend_report = call_llm(formatted_trend_prompt)

    return state.update(wikipedia_trend_report=wikipedia_trend_report)

@action(reads=["wikipedia_trend_report","trend_report"], writes=["merged_trend_report"])
def merge_trend_reports(state: State, file_path: Path) -> State:
    formatted_prompt = merge_trend_reports.format(trend_report = state['trend_report'],
                                                  context_report = state['wikipedia_trend_report'])
    merged_trend_report = call_llm(formatted_prompt)
    return state.update(merged_trend_report=merged_trend_report)

@action(reads=["wikipedia_trend_report","report_data"], writes=["executive_summary"])
def create_executive_summary(state: State, topic: str, focus: str) -> State:
    formatted_executive_prompt = executive_summary_prompt.format(topic=topic, focus=focus,
                                                         report=state['report_data']['report'],
                                                         trend_report=state['merged_trend_report'])

    executive_summary = call_llm(formatted_executive_prompt)
    return state.update(executive_summary=executive_summary)

In [ ]:
app = (
    ApplicationBuilder()
    .with_actions(
        load_report_from_file,
        create_trend_analysis,
        create_search_follow_up_document,
        search_wikipedia,
        vectorize_and_index_wikipedia,
        search_relevant_wiki_sections,
        create_wikipedia_trend_report,
        merge_trend_reports,
        create_executive_summary
    )
    .with_transitions(
        ("load_report_from_file", "create_trend_analysis"),
        ("create_trend_analysis", "create_search_follow_up_document"),
        ("create_search_follow_up_document", "search_wikipedia"),
        ("search_wikipedia", "vectorize_and_index_wikipedia"),
        ("vectorize_and_index_wikipedia", "search_relevant_wiki_sections"),
        ("search_relevant_wiki_sections","create_wikipedia_trend_report"),
        ("create_wikipedia_trend_report", "merge_trend_reports"),
        ("merge_trend_reports", "create_executive_summary"))
    .with_entrypoint("load_report_from_file")
    .with_tracker(
        "local",
        project=f"research_batch-single_run",
    )
    .build()
)

In [ ]:
app.visualize("./graph", format="png")

In [ ]:
formatted_trend_analysis_prompt = brief_analysis_prompt.format(topic=research_data['topic'],
                                                               focus=research_data['focus'],
                                                               report=research_data['report'],
                                                               sources=research_data['formatted_report_grounding'])

In [ ]:
trend_report = call_llm_flash(formatted_trend_analysis_prompt)

In [ ]:
formatted_follow_up_prompt = follow_up_context_prompt.format(topic=research_data['topic'],
                                                             focus=research_data['focus'],
                                                             report=research_data['report'],
                                                             trend_analysis=trend_report)

In [ ]:
follow_up_context = call_llm_flash(formatted_follow_up_prompt)

In [ ]:
different_followups = re.findall(r'(##[\s\S]+?)(?=##|$)', follow_up_context, flags=re.DOTALL)
follow_up_groups = dict()
for group in different_followups:
    group_name = group.split('\n')[0]
    followups = re.findall(r'[\*-]\s*(.+?)(?:\n|$)', group, flags=re.DOTALL)
    followups = [x.strip() for x in followups]
    follow_up_groups[group_name.strip()] = followups

In [ ]:
vectorized_sections = list()
for context_search in tqdm(context_data):
    context_topic = context_search['topic']
    context_focus = context_search['focus']
    context_query = context_search['query']
    for wiki_page in tqdm(context_search['context']):
        wiki_page_title = wiki_page['title']
        for section_idx, section in tqdm(enumerate(wiki_page['sections'])):
            section_copy = deepcopy(section)
            section_copy['vector'] = encoder.encode(f"{wiki_page_title}\n{section_copy['title']}\n{' '.join(section_copy['hierarchy_string'].split('|'))}\n{section_copy['text']}").tolist()

            section_copy['topic'] = context_topic
            section_copy['focus'] = context_focus
            section_copy['query'] = context_query
            section_copy['page_title'] = wiki_page_title
            section_copy['section_idx'] = section_idx

            vectorized_sections.append(section_copy)
            torch.mps.empty_cache()


In [ ]:
context_index_data = pd.DataFrame(vectorized_sections)

In [ ]:
context_index_data

In [ ]:
temp_research_table = index.create_table('temp_research', data=context_index_data, mode='overwrite')

In [ ]:
temp_research_table.create_fts_index(['text','title'], replace=True)

In [ ]:
context_results = dict()
all_context_documents = list()
for group, followup_group in follow_up_groups.items():
    all_search_results = list()
    for follow_up in followup_group:
        follow_up_vector = encoder.encode(follow_up).tolist()
        torch.mps.empty_cache()

        follow_up_search_vector = temp_research_table.search(follow_up_vector).distance_type('cosine').limit(20).to_pandas()
        follow_up_search_fts = temp_research_table.search(follow_up).limit(20).to_pandas()
        follow_up_search = pd.concat([follow_up_search_vector, follow_up_search_fts])
        all_search_results.append(follow_up_search)
    all_search_results = pd.concat(all_search_results)
    all_search_results = all_search_results.drop_duplicates(subset=['page_title','section_idx'], keep='first')
    all_search_results['section_start'] = all_search_results.groupby(['page_title','hierarchy_string'])['section_idx'].transform('min')
    all_search_results = all_search_results.sort_values(by=['page_title','section_start','section_idx'])
    context_results[group] = all_search_results
    all_context_documents.append(all_search_results)


all_context_documents = pd.concat(all_context_documents)
all_context_documents = all_context_documents.drop_duplicates(subset=['page_title','section_idx'], keep='first')
all_context_documents['section_start'] = all_context_documents.groupby(['page_title','hierarchy_string'])['section_idx'].transform('min')
all_context_documents = all_context_documents.sort_values(by=['page_title','section_start','section_idx'])

In [ ]:
all_context_documents = all_context_documents.drop_duplicates(subset=['page_title','text'])
all_context_documents = all_context_documents[all_context_documents['text'].str.strip() != '']

In [ ]:
formatted_context_sections = list()
all_context_grounding = list()
for _page_title, _group in all_context_documents.groupby(['page_title']):
    # group_text = '\n\n'.join(_group['text'])
    formatted_wiki_text = format_sections_w_citations(_group.to_dict(orient='records'), _page_title[0])
    group_text = ""
    for section in formatted_wiki_text:
        formatted_context_sections.append(deepcopy(section))
        _formatted_section = f"\n- {section['text']} ({section['citation']})"
        group_text += _formatted_section
    group_text = group_text.strip()
    all_context_grounding.append(f"#### Title: {_page_title[0]}\n\n{group_text}")

context_grounding = '\n\n'.join(all_context_grounding).strip()
print(len(context_grounding.split(' ')))
# all_context_grounding.append(context_grounding)

In [ ]:
areas_of_interest = list()
for follow_up_type, follow_up_points in follow_up_groups.items():
    points_to_follow_up = ['- ' + x for x in follow_up_points]
    points_to_follow_up = '\n'.join(points_to_follow_up)
    areas_of_interest.append(points_to_follow_up)

In [ ]:
formatted_prompts = list()
formatted_extraction_prompt = extract_context_information_prompt.format(topic=context_topic,
                                                                            focus=context_focus,
                                                                            areas_of_interest=_area_of_interest,
                                                                            trend_analysis=trend_report,
                                                                            articles=context_grounding)
formatted_prompts.append(formatted_extraction_prompt)

In [ ]:
follow_up_extractions = list()
for _follow_up_prompt in tqdm(formatted_prompts):
    extraction = call_llm_flash(_follow_up_prompt, max_tokens=7500)
    follow_up_extractions.append(extraction)

In [ ]:
all_follow_up_extractions = '\n\n----------\n\n'.join(follow_up_extractions)

In [ ]:
formatted_trend_interpolation_prompt = trend_interpolation_prompt.format(topic=context_topic,
                                                                         focus=context_focus,
                                                                         trend_analysis=trend_report,
                                                                         notes=all_follow_up_extractions)
len(formatted_extraction_prompt.split(' '))

In [ ]:
expanded_trend_report = call_llm_flash(formatted_trend_interpolation_prompt, max_tokens=10000)

In [ ]:
merged_trend_formatted_prompt = merged_trend_report_prompt.format(topic=context_topic,
                                                                  focus=context_focus,
                                                                  initial_report=trend_report,
                                                                  additional_report=expanded_trend_report)

In [ ]:
merged_trend_report = call_llm_flash(merged_trend_formatted_prompt, max_tokens=10000)

In [ ]:
full_research_folder = Path('wonky_data/data/full_research')
full_research_folder = full_research_folder.joinpath(research[0].stem)
full_research_folder.mkdir(parents=True, exist_ok=True)

full_research_documents_folder = full_research_folder.joinpath('documents')
full_research_documents_folder.mkdir(parents=True, exist_ok=True)

full_research_trace_folder = full_research_folder.joinpath('trace')
full_research_trace_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
report_file_name

In [ ]:
context_grounding_save_out = dict()
for _area_of_interest, _context in context_results.items():
    _cleaned_context = _context.copy()
    _cleaned_context['vector'] = _cleaned_context['vector'].apply(lambda x: x.tolist())
    context_grounding_save_out[_area_of_interest] = _cleaned_context.to_dict(orient='records')

In [ ]:
formatted_executive_summary_prompt = executive_summary_prompt.format(topic=research_data['topic'],
                                                                     focus=research_data['focus'],
                                                                     trend_report=merged_trend_report,
                                                                     report=research_data['report'])

In [ ]:
executive_summary = call_llm_flash(formatted_executive_summary_prompt, max_tokens=10000)

In [ ]:
trace_data = {"trend_report":trend_report,
                     "report":research_data['report'],
                     "follow_up_context":follow_up_context,
                     "expanded_trend_report":expanded_trend_report,
                     "trend_research":context_grounding_save_out,
                     "topic":research_data['topic'],
                     "focus":research_data['focus'],
                     "source_file_name":report_file_name,
                     "executive_summary":executive_summary,
              "followup_extractions":all_follow_up_extractions,
              "merged_trend_report":merged_trend_report}

documents_to_save = {"trend_report":trend_report,
                     "report":research_data['report'],
                     "follow_up_context":follow_up_context,
                     "expanded_trend_report":expanded_trend_report,
                     "source_file_name":report_file_name,
                     "executive_summary":executive_summary,
                     "followup_extractions":all_follow_up_extractions,
                     "merged_trend_report":merged_trend_report}

In [ ]:
with open(full_research_trace_folder.joinpath('research_data.json'),'w') as f:
    json.dump(trace_data,f)

In [ ]:
for document_type, document in documents_to_save.items():
    with open(full_research_documents_folder.joinpath(document_type+'.md'),'w') as f:
        f.write(document)

In [ ]:
print(f"topic: {trace_data['topic']} with a focus on: {trace_data['focus']}")